In [451]:
import pandas as pd
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [473]:
df = pd.read_csv("data/M.csv")

In [474]:
df.query("Estado.isnull()")

,Identificador,Fecha,Estado,Tipo,Cuenta,Beneficiario,Categoría,Importe,Divisa,Número,Notas
1446,2544,20/12/2022,NaN,Withdrawal,OPENBANK ECUENTA,Desconocido,Desconocido,-20.0,EUR,NaN,NaN


In [475]:
df.tail()

,Identificador,Fecha,Estado,Tipo,Cuenta,Beneficiario,Categoría,Importe,Divisa,Número,Notas
2187,2580,24/12/2022,R,Transfer,OPENBANK,METALICO,Transferencia,-5.75,EUR,#2580,NaN
2188,2610,25/12/2022,R,Transfer,OPENBANK,METALICO,Transferencia,-50.00,EUR,#2610,NaN
2189,981,07/07/2022,R,Transfer,OPENBANK,IBERCAJA,Transferencia,-29.27,EUR,NaN,
2190,1257,02/08/2022,R,Transfer,OPENBANK,IBERCAJA,Transferencia,-50.00,EUR,#1257,NaN
2191,232321,02/01/2023,R,Withdrawal,OPENBANK,ASISA DENTAL,Gastos Fijos:Salud,-7.99,EUR,NaN,NaN


In [ ]:
import pandas as pd

dataframe = pd.DataFrame({'Country': ['Afghanistan ', 'Afghanistan ', 'Brazil ', 'Brazil ', 'China ', 'China '],
                          'Year': [1999, 2000, 1999, 2000, 1999, 2000],
                          'Cases': [745.0, 2666.0, 37737.0, 80488.0, 212258.0, 213766.0],
                          'Population': ['19,987,071',  '20,595,360',  '172,006,362',  '174,504,898', '1,272,915,272', '1,280,428,583']})

display(dataframe)

dataframe.info()

(dataframe
    .assign(Cases = lambda dataset: pd.to_numeric(dataset.Cases, errors = "coerce").fillna(0))
    .info()
 )

In [501]:
def clean_data(df):
    return (df 
        .rename(columns = { 'Categoría' : 'Categoria' }) 
        .query("Tipo != 'Tranfer'")            
        .assign(Fecha = lambda dataset: pd.to_datetime(dataset.Fecha, format = "%d/%m/%Y"),
                Year = lambda dataset: dataset.Fecha.dt.year
        )
        .assign(
              Subcategoria = lambda dataset : dataset.Categoria.str.split(':').str.get(1),
              Categoria = lambda dataset : dataset.Categoria.str.split(':').str.get(0),
              Tipo = lambda df: df.Categoria.map(lambda x: "Ingresos" if "Ingresos" in x else "Gastos"),
              Beneficiario = lambda df: df.Beneficiario.fillna("No Definido"),
              Year = lambda dataset: pd.Categorical(dataset.Year, 
                                            categories = sorted(dataset.Year.value_counts().index.to_list()),
                                            ordered = True)
               )
        .query("Estado == 'R'")
        .filter(["Fecha", "Year", "Tipo", "Categoria", "Subcategoria", "Beneficiario", "Importe"])
        .convert_dtypes()
      )

In [502]:
df = pd.read_csv("data/M.csv").pipe(clean_data)

In [503]:

df.sort_values("Year")

,Fecha,Year,Tipo,Categoria,Subcategoria,Beneficiario,Importe
0,2022-01-11,2022,Ingresos,Ingresos,Intereses Bancarios,OPENBANK,0.69
1470,2022-01-04,2022,Gastos,Extras,Regalos,FNAC,-24.99
1469,2022-01-04,2022,Gastos,Ocio,Restaurantes,RODILLA,-2.5
1468,2022-01-04,2022,Gastos,Ocio,Restaurantes,RODILLA,-6.7
1467,2022-01-04,2022,Gastos,Despacho - Gastos fijos,Cuota Colegio Abogados,ICAM,-157.14
...,...,...,...,...,...,...,...
731,2022-03-16,2022,Gastos,Supervivencia,Holden,AMAZON,-13.99
732,2022-03-16,2022,Gastos,Extras,Casa,AMAZON,-20.99
733,2022-03-17,2022,Gastos,Supervivencia,Alimentación,DIA,-23.31
727,2022-03-13,2022,Gastos,Supervivencia,Alimentación,DIA,-23.07


In [528]:
def clean_data(df):
    return (df 
        .rename(columns = { 'Categoría' : 'Categoria' }) 
        .query("Tipo != 'Tranfer'")            
        .assign(Fecha = lambda dataset: pd.to_datetime(dataset.Fecha, format = "%d/%m/%Y"),
                Year = lambda dataset: dataset.Fecha.dt.year.astype(str),
                Subcategoria = lambda dataset : dataset.Categoria.str.split(':').str.get(1),
                Categoria = lambda dataset : dataset.Categoria.str.split(':').str.get(0),
                Tipo = lambda df: df.Categoria.map(lambda x: "Ingresos" if "Ingresos" in x else "Gastos"),
                Beneficiario = lambda df: df.Beneficiario.fillna("No Definido")
          )
        .query("Estado == 'R'")
        .filter(["Fecha", "Year", "Tipo", "Categoria", "Subcategoria", "Beneficiario", "Importe"])
      )



def return_despacho_movements(df):
        return ( df
            .query("Categoria.str.startswith('Despacho')") 
            .assign(Categoria = lambda df: df.Categoria.str[11:]) 
            .convert_dtypes()
               )


def return_hogar_movements(df):
        return (df 
          .assign(
            Beneficiario = lambda df: df.apply(lambda dataset: "No Definido" if dataset.Categoria.startswith("Despacho") else dataset.Beneficiario, axis = 'columns'), 
            Subcategoria = lambda df: df.apply(lambda dataset: "Despacho" if dataset.Categoria.startswith("Despacho") else dataset.Subcategoria, axis = 'columns'), 
            Tipo = lambda df: df.apply(lambda dataset: "Ingresos" if dataset.Categoria.startswith("Despacho") else dataset.Tipo, axis = 'columns'),
            Categoria = lambda df: df.Categoria.map(lambda x: "Ingresos" if x.startswith("Despacho") else x)
          )
            .convert_dtypes()
               )


def total_rows(df, columns):
    if len(columns) > 1:
        group_by_columns = columns[:-1]
        first_column = group_by_columns[-1]
        last_column = columns[-1]

        df_totales = df \
            .groupby(group_by_columns).sum().reset_index() \
            .query(first_column + " not in ['Total']")

        df_totales[last_column] = df_totales[first_column].map(lambda x: "Total" if x == "Total" else "Total " + x)
    else:
        last_column = columns[0]
        df_totales = df.sum(numeric_only=True).to_frame().transpose()
        df_totales[last_column] = "Total"

    return df_totales


def get_months():
    return {
        1 : 'Enero',
        2 : 'Febrero',
        3 : 'Marzo',
        4 : 'Abril',
        5 : 'Mayo',
        6 : 'Junio',
        7 : 'Julio',
        8 : 'Agosto',
        9 : 'Septiembre',
        10 : 'Octubre',
        11 : 'Noviembre',
        12 : 'Diciembre'
      }


def clean_pivot_table(df, left_columns, top_columns, value_column,  aggfunc_name="sum"):
    return (df
            .pivot_table(index =left_columns,
                       columns = top_columns,
                       values=value_column,
                       aggfunc=aggfunc_name,
                       fill_value = 0)
            .reset_index()
            .rename_axis(columns = None)
 )


def get_average(df):
    last_month = 12
    while df[last_month] == 0:
        last_month -= 1

    total = 0
    for month in range(last_month):
        total += df[month + 1]
        
    return total / last_month


def pivot_by_category(df, category):
    group_list = ["Month"]
    group_list.extend(category)

    fist_item = category[0]

    return (
        df
         .assign(Month = lambda dataset: dataset.Fecha.dt.month)
         .groupby( group_list, as_index = False).Importe.sum()
         .pipe(clean_pivot_table, left_columns = category, top_columns = "Month", value_column = "Importe")
         .assign(
             Total = lambda dataset: dataset.filter(range(13)).sum(axis='columns'),
             Media = lambda dataset: dataset.apply(lambda df: get_average(df), axis = 'columns')
         )
        .rename(columns = get_months())
        .reset_index(drop = True)
        .fillna({ fist_item : "Total" })
        .rename_axis(None, axis='columns')
        .round(2)
    )


def pivot_by_category_totals(df, columns):
    df_pivot = df.pipe(pivot_by_category, columns)
    df_all_totales = df_pivot
    for n in range(0,len(columns)):
        df_all_totales = pd.concat([df_all_totales, df_pivot.pipe(total_rows, columns[:n + 1])])

    for n in range(0,len(columns)):
        df_all_values = df_all_totales[columns[n]].value_counts().index.to_list()
        df_all_values_total = list(filter(lambda x: x.startswith("Total"), df_all_values))
        df_all_values = list(filter(lambda x: not x.startswith("Total"), df_all_values))
        df_all_values = sorted(df_all_values)
        df_all_values.extend(sorted(df_all_values_total))

        df_all_totales[columns[n]] = pd.Categorical(
            df_all_totales[columns[n]],
            categories=df_all_values,
            ordered=True)

    return df_all_totales.sort_values(columns) \
                    .reset_index(drop = True)



In [529]:
df = pd.read_csv("data/M.csv").pipe(clean_data)
df_despacho = df.pipe(return_despacho_movements)
df_hogar = df.pipe(return_hogar_movements)

In [530]:
df_hogar.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2191 entries, 0 to 2191
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   Fecha         2191 non-null   datetime64[ns]
 1   Year          2191 non-null   string        
 2   Tipo          2191 non-null   string        
 3   Categoria     2191 non-null   string        
 4   Subcategoria  2001 non-null   string        
 5   Beneficiario  2191 non-null   string        
 6   Importe       2191 non-null   Float64       
dtypes: Float64(1), datetime64[ns](1), string(5)
memory usage: 139.1 KB


In [533]:
pivot_by_category_totals(df_hogar, ["Year", "Tipo"])

,Year,Tipo,Enero,Febrero,Marzo,Abril,Mayo,Junio,Julio,Agosto,Septiembre,Octubre,Noviembre,Diciembre,Total,Media
0,2022,Gastos,-4858.84,-4370.48,-5208.63,-5021.79,-5210.0,-7771.75,-8288.79,-3461.1,-5234.44,-12485.53,-8505.44,-9069.84,-79486.63,-6623.89
1,2022,Ingresos,2642.34,9696.62,7735.52,5602.14,5627.59,9110.07,5291.84,2587.06,2727.39,9266.42,10296.45,11082.86,81666.30,6805.52
2,2022,Total 2022,-2216.5,5326.14,2526.89,580.35,417.59,1338.32,-2996.95,-874.04,-2507.05,-3219.11,1791.01,2013.02,2179.67,181.63
3,2023,Gastos,-7.99,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-7.99,-7.99
4,2023,Total 2023,-7.99,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-7.99,-7.99
5,Total,NaN,-2224.49,5326.14,2526.89,580.35,417.59,1338.32,-2996.95,-874.04,-2507.05,-3219.11,1791.01,2013.02,2171.68,173.64
